hasyv2/hasyv2/hasy-data

In [7]:
!pip install torch torchvision pillow numpy pandas scikit-learn opencv-python tqdm

In [9]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.amp import GradScaler, autocast
import cv2
from tqdm import tqdm

# --- Кастомный датасет ---
class HASYv2Dataset(Dataset):
    def __init__(self, df, images_dir, transform=None):
        self.df = df
        self.images_dir = images_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.basename(self.df.iloc[idx]['path'])
        img_path = os.path.join(self.images_dir, img_name)
        try:
            img = Image.open(img_path).convert('L')
        except FileNotFoundError:
            print(f"Файл не найден: {img_path}")
            raise
        label = self.df.iloc[idx]['normalized_symbol_id']
        if self.transform:
            img = self.transform(img)
        return img, label

# --- Улучшенная CNN ---
class EnhancedSymbolCNN(nn.Module):
    def __init__(self, num_classes=369):
        super(EnhancedSymbolCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# --- Тест форматирования ---
def test_formatting(dataset, output_dir='test_formatted'):
    os.makedirs(output_dir, exist_ok=True)
    for i in range(min(5, len(dataset))):
        # Получаем исходное изображение без трансформации
        img_name = os.path.basename(dataset.df.iloc[i]['path'])
        img_path = os.path.join(dataset.images_dir, img_name)
        img = Image.open(img_path).convert('L')
        label = dataset.df.iloc[i]['normalized_symbol_id']

        # Применяем бинаризацию и инверсию
        img_np = np.array(img)
        _, img_np = cv2.threshold(img_np, 128, 255, cv2.THRESH_BINARY)
        img_np = 255 - img_np  # Инверсия для черного фона и белого символа
        img_pil = Image.fromarray(img_np.astype(np.uint8))
        img_pil.save(os.path.join(output_dir, f'test_{i}_label_{label}.png'))
    print(f"✅ Тестовые изображения сохранены в {output_dir}")

# --- Основная функция обучения ---
def main():
    # Загрузка CSV
    csv_path = 'hasyv2/hasyv2/hasy-data-labels.csv'
    df = pd.read_csv(csv_path)

    # Нормализация symbol_id
    unique_symbol_ids = sorted(df['symbol_id'].unique())
    id_to_normalized = {old_id: new_id for new_id, old_id in enumerate(unique_symbol_ids)}
    if len(unique_symbol_ids) != 369:
        raise ValueError(f"Ожидалось 369 уникальных классов, найдено {len(unique_symbol_ids)}")
    df['normalized_symbol_id'] = df['symbol_id'].map(id_to_normalized)
    print(f"Нормализованный диапазон symbol_id: [{df['normalized_symbol_id'].min()}, {df['normalized_symbol_id'].max()}]")
    print(f"Количество уникальных классов: {len(df['normalized_symbol_id'].unique())}")

    # Обновление словаря class_to_symbol
    with open('hasyv2/hasyv2/symbols.csv', 'r', encoding='utf-8') as f:
        symbols_df = pd.read_csv(f)
    class_to_symbol = {}
    for index, row in symbols_df.iterrows():
        class_id = row['symbol_id']
        latex_command = row['latex']
        class_to_symbol[str(id_to_normalized[class_id])] = latex_command
    with open('model/class_to_symbol_normalized.json', 'w', encoding='utf-8') as f:
        json.dump(class_to_symbol, f, ensure_ascii=False, indent=4)
    print("✅ Словарь сохранён в model/class_to_symbol_normalized.json")

    # Разделение данных
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    images_dir = 'hasyv2/hasyv2/hasy-data'

    # Аугментация данных с небелыми фонами
    train_transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1),  # Нерегулярные фоны
        transforms.RandomGrayscale(p=0.5),  # Случайно серые изображения
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
        transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: 1 - x if torch.rand(1).item() > 0.5 else x),  # Случайная инверсия после ToTensor
        transforms.Normalize(mean=[0.5], std=[0.5]),
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.1)),
    ])
    test_transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
    ])

    # Создание датасетов
    train_dataset = HASYv2Dataset(train_df, images_dir, transform=train_transform)
    test_dataset = HASYv2Dataset(test_df, images_dir, transform=test_transform)

    # Тест форматирования
    test_formatting(train_dataset)

    # Вычисление классовых весов
    class_weights = compute_class_weight('balanced', classes=np.unique(df['normalized_symbol_id']),
                                        y=df['normalized_symbol_id'])
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    # Создание загрузчиков
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)
    print(f"✅ Данные загружены: {len(train_dataset)} обучающих, {len(test_dataset)} тестовых образцов")

    # Инициализация модели
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = EnhancedSymbolCNN(num_classes=369).to(device)
    print(f"✅ Модель на {device}")

    # Загрузка предобученной модели, если существует
    model_path = 'models/hasyv2_model_best.pth'
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
        print(f"✅ Загружена предобученная модель из {model_path}")

    # Оптимизатор и функция потерь
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scaler = GradScaler()

    # Обучение
    num_epochs = 20
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch')
        for images, labels in progress_bar:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item()
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
        avg_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

        # Оценка
        model.eval()
        correct = 0
        total = 0
        top5_correct = 0
        with torch.no_grad():
            for images, labels in tqdm(test_loader, desc='Evaluating', unit='batch'):
                images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                    outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                # Top-5 accuracy
                _, top5_pred = outputs.topk(5, dim=1)
                top5_correct += top5_pred.eq(labels.view(-1, 1).expand_as(top5_pred)).sum().item()
        accuracy = correct / total
        top5_accuracy = top5_correct / total
        print(f'Точность на тестовой выборке: {accuracy:.4f} ({correct}/{total}), Top-5 точность: {top5_accuracy:.4f}')

        # Сохранение модели после каждой эпохи
        torch.save(model.state_dict(), f'model/hasyv2_model_epoch_{epoch + 1}.pth')
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'model/hasyv2_model_arg.pth')
            print(f"✅ Сохранена лучшая модель с точностью {accuracy:.4f}")

    print(f"✅ Обучение завершено. Лучшая точность: {best_accuracy:.4f}, Лучшая Top-5 точность: {top5_accuracy:.4f}")

if __name__ == "__main__":
    main()

Нормализованный диапазон symbol_id: [0, 368]
Количество уникальных классов: 369
✅ Словарь сохранён в model/class_to_symbol_normalized.json
✅ Тестовые изображения сохранены в test_formatted
✅ Данные загружены: 134586 обучающих, 33647 тестовых образцов
✅ Модель на cuda
✅ Загружена предобученная модель из models/hasyv2_model_best.pth


Epoch 1/20: 100%|███████████████████████████████████████████████████| 2103/2103 [00:26<00:00, 80.33batch/s, loss=2.4932]


Epoch [1/20], Average Loss: 2.0726


Evaluating: 100%|█████████████████████████████████████████████████████████████████| 526/526 [00:02<00:00, 191.03batch/s]


Точность на тестовой выборке: 0.6531 (21976/33647), Top-5 точность: 0.9433
✅ Сохранена лучшая модель с точностью 0.6531


...

Точность на тестовой выборке: 0.6819 (22944/33647), Top-5 точность: 0.9617


Epoch 20/20: 100%|██████████████████████████████████████████████████| 2103/2103 [00:26<00:00, 79.10batch/s, loss=1.2985]


Epoch [20/20], Average Loss: 1.5103


Evaluating: 100%|█████████████████████████████████████████████████████████████████| 526/526 [00:02<00:00, 190.19batch/s]

Точность на тестовой выборке: 0.6857 (23071/33647), Top-5 точность: 0.9595
✅ Обучение завершено. Лучшая точность: 0.6993, Лучшая Top-5 точность: 0.9595


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# y_true — реальные метки, y_pred — предсказания
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='viridis', values_format='d')
plt.show()
